In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import os

In [2]:
# read in CSV
hmb_1_imu = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/imu-data.csv')
hmb_1_imu.isnull().sum()

Time                                0
header.seq                          0
header.stamp.secs                   0
header.stamp.nsecs                  0
header.frame_id                     0
orientation.x                       0
orientation.y                       0
orientation.z                       0
orientation.w                       0
orientation_covariance_0            0
orientation_covariance_1            0
orientation_covariance_2            0
orientation_covariance_3            0
orientation_covariance_4            0
orientation_covariance_5            0
orientation_covariance_6            0
orientation_covariance_7            0
orientation_covariance_8            0
angular_velocity.x                  0
angular_velocity.y                  0
angular_velocity.z                  0
angular_velocity_covariance_0       0
angular_velocity_covariance_1       0
angular_velocity_covariance_2       0
angular_velocity_covariance_3       0
angular_velocity_covariance_4       0
angular_velo

In [3]:
def attach_image(dataframe,images):

    # Attatching image files to the inputs
    
    image_base_dir = f'./Data/ROSBAG/Ch2_002/images/{images}'
    
    # Define camera folders
    camera_folders = {
        "left": os.path.join(image_base_dir, "left"),
        "center": os.path.join(image_base_dir, "center"),
        "right": os.path.join(image_base_dir, "right"),
    }
    
    # Add new columns for image paths
    for camera in camera_folders:
        dataframe[f"{camera}_image_path"] = dataframe['Time'].apply(
            lambda x: os.path.join(camera_folders[camera], f"frame_{x}.png")
        )
    return dataframe
imu_image = attach_image(hmb_1_imu,'HMB_1')
imu_image.head()    

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,orientation.x,orientation.y,orientation.z,orientation.w,orientation_covariance_0,...,linear_acceleration_covariance_2,linear_acceleration_covariance_3,linear_acceleration_covariance_4,linear_acceleration_covariance_5,linear_acceleration_covariance_6,linear_acceleration_covariance_7,linear_acceleration_covariance_8,left_image_path,center_image_path,right_image_path
0,1.479424e+09,1183922,1479424214,778361082,/imu,0.012995,0.028516,0.919373,0.392138,0.017453,...,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...
1,1.479424e+09,1183923,1479424214,780781030,/imu,0.012959,0.028632,0.919347,0.392191,0.017453,...,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...
2,1.479424e+09,1183924,1479424214,783006906,/imu,0.012934,0.028753,0.919331,0.392220,0.017453,...,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...
3,1.479424e+09,1183925,1479424214,784673929,/imu,0.012914,0.028845,0.919311,0.392260,0.017453,...,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...
4,1.479424e+09,1183926,1479424214,785916090,/imu,0.012933,0.028897,0.919291,0.392304,0.017453,...,0.0,0.0,0.0004,0.0,0.0,0.0,0.0004,./Data/ROSBAG/Ch2_002/images/HMB_1\left\frame_...,./Data/ROSBAG/Ch2_002/images/HMB_1\center\fram...,./Data/ROSBAG/Ch2_002/images/HMB_1\right\frame...


In [4]:
steer = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/vehicle-steering_report.csv')
throttle = pd.read_csv('./Data/ROSBAG/Ch2_002/HMB_1/vehicle-throttle_info_report.csv')

In [5]:
steer.shape

(11040, 17)

In [6]:
steer.isnull().sum()

Time                            0
header.seq                      0
header.stamp.secs               0
header.stamp.nsecs              0
header.frame_id             11040
steering_wheel_angle            0
steering_wheel_angle_cmd        0
steering_wheel_torque           0
speed                           0
enabled                         0
override                        0
driver                          0
fault_wdc                       0
fault_bus1                      0
fault_bus2                      0
fault_calibration               0
fault_connector                 0
dtype: int64

In [7]:
throttle.shape

(22118, 8)

In [8]:
throttle.isnull().sum()

Time                      0
header.seq                0
header.stamp.secs         0
header.stamp.nsecs        0
header.frame_id       22118
throttle_pc               0
throttle_rate             0
engine_rpm                0
dtype: int64

In [9]:
throttle_steer = steer.merge(throttle,on = 'Time',how = 'left')
throttle_steer.head()

,Time,header.seq_x,header.stamp.secs_x,header.stamp.nsecs_x,header.frame_id_x,steering_wheel_angle,steering_wheel_angle_cmd,steering_wheel_torque,speed,enabled,...,fault_bus2,fault_calibration,fault_connector,header.seq_y,header.stamp.secs_y,header.stamp.nsecs_y,header.frame_id_y,throttle_pc,throttle_rate,engine_rpm
0,1.479424e+09,278688,1479424214,787219361,NaN,0.003491,0.0,0.1250,22.880556,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.479424e+09,278689,1479424214,807231859,NaN,0.003491,0.0,0.1250,22.886110,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.479424e+09,278690,1479424214,827300730,NaN,0.003491,0.0,0.1250,22.891666,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.479424e+09,278691,1479424214,847355643,NaN,0.003491,0.0,0.1875,22.897223,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.479424e+09,278692,1479424214,867355824,NaN,0.003491,0.0,0.0000,22.902779,False,...,False,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
throttle_steer.isnull().sum()

Time                            0
header.seq_x                    0
header.stamp.secs_x             0
header.stamp.nsecs_x            0
header.frame_id_x           11040
steering_wheel_angle            0
steering_wheel_angle_cmd        0
steering_wheel_torque           0
speed                           0
enabled                         0
override                        0
driver                          0
fault_wdc                       0
fault_bus1                      0
fault_bus2                      0
fault_calibration               0
fault_connector                 0
header.seq_y                11040
header.stamp.secs_y         11040
header.stamp.nsecs_y        11040
header.frame_id_y           11040
throttle_pc                 11040
throttle_rate               11040
engine_rpm                  11040
dtype: int64

### Merging both the throttle and steer dataframes shows that there are no matching time frames for both datasets 

In [11]:
steer = attach_image(steer,'HMB_1')
throttle = attach_image(throttle,'HMB_1')

In [12]:
steer = steer.drop(columns = ['header.seq','header.stamp.secs','header.stamp.nsecs','header.frame_id','steering_wheel_angle_cmd',
                              'enabled','override','driver','fault_wdc','fault_bus1','fault_bus2','fault_calibration','fault_connector'])
throttle = throttle.drop(columns = ['header.seq','header.stamp.secs','header.stamp.nsecs','header.frame_id',])
steer.to_csv('./cleaned_data/steer_data_HMB_1.csv', index = False)
throttle.to_csv('./cleaned_data/throttle_data_HMB_1.csv', index = False)


### Clean the sterring and throttle data

In [13]:
# Check the image size. Assume each image is the same since it came from the same car.
img = Image.open('./Data/ROSBAG/Ch2_002/images/HMB_1/left/frame_1479424215877601572.png')
width, height = img.size
print(f"Image size: {width}x{height} (Color image)")

Image size: 640x480 (Color image)


In [16]:
def prepare_data_with_labels(df, image_columns, label_columns, target_size=(224, 224)):
    """
    Prepares X (images) and y (labels) for training a CNN.

    Parameters:
        df (pd.DataFrame): DataFrame containing image paths and labels.
        image_columns (list): List of column names with image paths (e.g., ['left_image_path', 'center_image_path', 'right_image_path']).
        label_columns (list): List of column names for labels (e.g., ['steering_angle', 'throttle']).
        target_size (tuple): Target size for resizing images (width, height).

    Returns:
        np.ndarray: Array of image data (X).
        np.ndarray: Array of labels (y).
    """
    X_images = []
    y_labels = []
    
    for _, row in df.iterrows():
        row_images = []
        
        # Process images for the current row
        for col in image_columns:
            image_path = row[col]
            
            if os.path.exists(image_path):  # Ensure file exists
                try:
                    img = Image.open(image_path).resize(target_size)  # Open and resize
                    img_array = np.array(img) / 255.0  # Normalize
                    row_images.append(img_array)
                except Exception as e:
                    print(f"Error loading image {image_path}: {e}")
            else:
                print(f"Image not found: {image_path}")
        
        # Check if all images for the row are processed
        if len(row_images) == len(image_columns):
            # Combine images if using multiple cameras (optional)
            combined_images = np.stack(row_images, axis=-1)  # Shape: (height, width, 3 * number of cameras)
            X_images.append(combined_images)
            
            # Collect the corresponding labels
            y_labels.append(row[label_columns].values)
    
    # Convert to NumPy arrays
    X = np.array(X_images, dtype=np.float32)
    y = np.array(y_labels, dtype=np.float32)
    
    return X, y
